In [ ]:
try:
    import resnest
except ModuleNotFoundError:
    !pip install -q "../input/resnest50-fast-package/resnest-0.0.6b20200701/resnest"

try:
    import efficientnet_pytorch
except ModuleNotFoundError:
    !pip install -q "../input/efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master"

In [ ]:
import re
import sys 
import time
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')

import cv2
from efficientnet_pytorch import EfficientNet
import librosa as lb
import numpy as np
import pandas as pd
from pathlib import Path
from resnest.torch import resnest50
import soundfile as sf
import timm
import torch
from torch import nn
from torch.nn import functional as F
from  torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm

In [ ]:
NUM_CLASSES = 397
SR = 32_000
DURATION = 5
THRESH = 0.12
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("DEVICE:", DEVICE)
TEST_AUDIO_ROOT = Path("../input/birdclef-2021/test_soundscapes")
SAMPLE_SUB_PATH = "../input/birdclef-2021/sample_submission.csv"
TARGET_PATH = None

if not len(list(TEST_AUDIO_ROOT.glob("*.ogg"))):
    TEST_AUDIO_ROOT = Path("../input/birdclef-2021/train_soundscapes")
    SAMPLE_SUB_PATH = None
    # SAMPLE_SUB_PATH = "../input/birdclef-2021/sample_submission.csv"
    TARGET_PATH = Path("../input/birdclef-2021/train_soundscape_labels.csv")

In [ ]:
class MelSpecComputer:
    def __init__(self, sr, n_mels, fmin, fmax, **kwargs):
        self.sr = sr
        self.n_mels = n_mels
        self.fmin = fmin
        self.fmax = fmax
        kwargs["n_fft"] = kwargs.get("n_fft", self.sr//10)
        kwargs["hop_length"] = kwargs.get("hop_length", self.sr//(10*4))
        self.kwargs = kwargs

    def __call__(self, y):
        melspec = lb.feature.melspectrogram(
            y, sr=self.sr, n_mels=self.n_mels, fmin=self.fmin, fmax=self.fmax, **self.kwargs,
        )
        melspec = lb.power_to_db(melspec).astype(np.float32)
        return melspec


def mono_to_color(X, eps=1e-6, mean=None, std=None):
    mean = mean or X.mean()
    std = std or X.std()
    X = (X - mean) / (std + eps)
    
    _min, _max = X.min(), X.max()

    if (_max - _min) > eps:
        V = np.clip(X, _min, _max)
        V = 255 * (V - _min) / (_max - _min)
        V = V.astype(np.uint8)
    else:
        V = np.zeros_like(X, dtype=np.uint8)

    return V


def crop_or_pad(y, length):
    if len(y) < length:
        y = np.concatenate([y, length - np.zeros(len(y))])
    elif len(y) > length:
        y = y[:length]
    return y


class BirdCLEFDataset(Dataset):
    def __init__(self, data, sr=SR, n_mels=128, fmin=0, fmax=None, duration=DURATION, step=None, res_type="kaiser_fast", resample=True):
        
        self.data = data
        
        self.sr = sr
        self.n_mels = n_mels
        self.fmin = fmin
        self.fmax = fmax or self.sr//2

        self.duration = duration
        self.audio_length = self.duration*self.sr
        self.step = step or self.audio_length
        
        self.res_type = res_type
        self.resample = resample

        self.mel_spec_computer = MelSpecComputer(sr=self.sr, n_mels=self.n_mels, fmin=self.fmin,
                                                 fmax=self.fmax)
    def __len__(self):
        return len(self.data)
    
    @staticmethod
    def normalize(image):
        image = image.astype("float32", copy=False) / 255.0
        image = np.stack([image, image, image])
        return image
    
    def audio_to_image(self, audio):
        melspec = self.mel_spec_computer(audio)
        
        image = mono_to_color(melspec)
        image = self.normalize(image)
        return image

    def read_file(self, filepath):
        audio, orig_sr = sf.read(filepath, dtype="float32")

        if self.resample and orig_sr != self.sr:
            audio = lb.resample(audio, orig_sr, self.sr, res_type=self.res_type)
          
        audios = []
        for i in range(self.audio_length, len(audio) + self.step, self.step):
            start = max(0, i - self.audio_length)
            end = start + self.audio_length
            audios.append(audio[start:end])
            
        if len(audios[-1]) < self.audio_length:
            audios = audios[:-1]
            
        images = [self.audio_to_image(audio) for audio in audios]
        images = np.stack(images)
        
        return images
        
    def __getitem__(self, idx):
        return self.read_file(self.data.loc[idx, "filepath"])


class densenet(nn.Module):
    def __init__(self, n_class=397, model_name='densenet121', pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained,num_classes=0)
        self.dropout = torch.nn.Dropout(p=0.2)
        self.classifier = nn.Linear(1024, n_class)

    def forward(self, x):
        x = self.model(x)
        x = self.dropout(x)
        x = self.classifier(x)
        return x


class nfnet(nn.Module):
    def __init__(self, n_class=397, model_name='eca_nfnet_l0', pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained,num_classes=0)
        self.dropout = torch.nn.Dropout(p=0.4)
        self.classifier = nn.Linear(2304, n_class)

    def forward(self, x):
        x = self.model(x)
        x = self.dropout(x)
        x = self.classifier(x)
        
        return x


def load_densenet(checkpoint_path, num_classes=NUM_CLASSES):
    net = densenet(pretrained=False)
    dummy_device = torch.device("cpu")
    d = torch.load(checkpoint_path, map_location=dummy_device)
#     for key in list(d.keys()):
#         d[key.replace("model.", "")] = d.pop(key)
    net.load_state_dict(d)
    net = net.to(DEVICE)
    net = net.eval()
    return net


def load_resnest50dnet(checkpoint_path, num_classes=NUM_CLASSES):
    net = timm.create_model('resnest50d',pretrained=False)
    net.fc = nn.Linear(net.fc.in_features, num_classes)
    dummy_device = torch.device("cpu")
    d = torch.load(checkpoint_path, map_location=dummy_device)
    for key in list(d.keys()):
        d[key.replace("model.", "")] = d.pop(key)
    net.load_state_dict(d)
    net = net.to(DEVICE)
    net = net.eval()
    return net


def load_resnestnet(checkpoint_path, num_classes=NUM_CLASSES):
    net = resnest50(pretrained=False)
    net.fc = nn.Linear(net.fc.in_features, num_classes)
    dummy_device = torch.device("cpu")
    d = torch.load(checkpoint_path, map_location=dummy_device)
    for key in list(d.keys()):
        d[key.replace("model.", "")] = d.pop(key)
    net.load_state_dict(d)
    net = net.to(DEVICE)
    net = net.eval()
    return net


def load_nfnet(checkpoint_path, num_classes=NUM_CLASSES):
    net = nfnet(pretrained=False)
    dummy_device = torch.device("cpu")
    d = torch.load(checkpoint_path, map_location=dummy_device)
#     for key in list(d.keys()):
#         d[key.replace("model.", "")] = d.pop(key)
    net.load_state_dict(d)
    net = net.to(DEVICE)
    net = net.eval()
    return net


def load_efficient_net(checkpoint_path, num_classes=NUM_CLASSES):
    net = EfficientNet.from_name('efficientnet-b5')
    net._fc = nn.Linear(net._fc.in_features, num_classes)
    #model._fc = nn.Linear(in_features=model._fc.in_features, out_features=6)
    dummy_device = torch.device("cpu")
    d = torch.load(checkpoint_path, map_location=dummy_device)
    for key in list(d.keys()):
        d[key.replace("model.", "")] = d.pop(key)
    #model.load_state_dict(torch.load('../input/pytorch-efficientnet/best_model.pth'))
    net.load_state_dict(d)
    net = net.to(DEVICE)
    net = net.eval()
    return net


@torch.no_grad()
def get_thresh_preds(out, thresh=None):
    thresh = thresh or THRESH
    o = (-out).argsort(1)
    npreds = (out > thresh).sum(1)
    preds = []
    for oo, npred in zip(o, npreds):
        preds.append(oo[:npred].cpu().numpy().tolist())
    return preds


def get_bird_names(preds):
    bird_names = []
    for pred in preds:
        if not pred:
            bird_names.append("nocall")
        else:
            bird_names.append(" ".join([INV_LABEL_IDS[bird_id] for bird_id in pred]))
    return bird_names


def predict(nets, test_data, names=True):
    preds = []
    x = [0.04, 0.05, 0.05]
    with torch.no_grad():
        for idx in  tqdm(list(range(len(test_data)))):
            xb = torch.from_numpy(test_data[idx]).to(DEVICE)
            pred = 0.
            for model_id, net in enumerate(nets):
                o = net(xb)
                o = torch.sigmoid(o)
                if model_id == 0:
                    pred += o * (1 - x[0] * 5 - x[1] * 4 - x[2])
                elif model_id in (1, 2, 3, 4, 5):
                    pred += o * x[0]
                elif model_id in (6, 7, 8, 9):
                    pred += o * x[1]
                elif model_id == 10:
                    pred += o * x[2]

            pred /= 1
            
            if names:
                pred = get_bird_names(get_thresh_preds(pred))

            preds.append(pred)
    return preds


def preds_as_df(data, preds):
    sub = {
        "row_id": [],
        "birds": [],
    }
    
    for row, pred in zip(data.itertuples(False), preds):
        row_id = [f"{row.id}_{row.site}_{5*i}" for i in range(1, len(pred)+1)]
        sub["birds"] += pred
        sub["row_id"] += row_id
        
    sub = pd.DataFrame(sub)
    
    if SAMPLE_SUB_PATH:
        sample_sub = pd.read_csv(SAMPLE_SUB_PATH, usecols=["row_id"])
        sub = sample_sub.merge(sub, on="row_id", how="left")
        sub["birds"] = sub["birds"].fillna("nocall")
    return sub

In [ ]:
data = pd.DataFrame(
     [(path.stem, *path.stem.split("_"), path) for path in Path(TEST_AUDIO_ROOT).glob("*.ogg")],
    columns = ["filename", "id", "site", "date", "filepath"]
)
df_train = pd.read_csv("../input/birdclef-2021/train_metadata.csv")
LABEL_IDS = {label: label_id for label_id,label in enumerate(sorted(df_train["primary_label"].unique()))}
INV_LABEL_IDS = {val: key for key,val in LABEL_IDS.items()}
test_data = BirdCLEFDataset(data=data)

checkpoint_paths = [
    Path("../input/kkiller-birdclef-models-public/birdclef_resnest50_fold0_epoch_10_f1_val_06471_20210417161101.pth"),
    Path("../input/densenet/birdclef_densenet121_fold0_epoch_16_f1_val_06185_20210516040431.pth"),
    Path("../input/densenet/birdclef_densenet121_fold1_epoch_17_f1_val_06234_20210516072613.pth"),
    Path("../input/densenet/birdclef_densenet121_fold2_epoch_16_f1_val_06259_20210517151626.pth"),
    Path("../input/densenet/birdclef_densenet121_fold3_epoch_16_f1_val_06143_20210517180831.pth"),
    Path("../input/densenet/birdclef_densenet121_fold4_epoch_16_f1_val_06165_20210517210136.pth"),
    Path('../input/bridclef-resnest50-weight/birdclef_resnest50_fold4_epoch_29_f1_val_07694_20210513205331.pth'),
    Path('../input/bridclef-resnest50-weight/birdclef_resnest50_fold1_epoch_18_f1_val_07636_20210512152028.pth'),
    Path('../input/bridclef-resnest50-weight/birdclef_resnest50_fold2_epoch_24_f1_val_07728_20210512202556.pth'),
    Path('../input/bridclef-resnest50-weight/birdclef_resnest50_fold3_epoch_28_f1_val_07609_20210513105835.pth'),
    Path("../input/birdclefefficientnetb5-fold0-epoch-09-10/birdclef_efficientnet-b5_fold0_epoch_09_f1_val_07398_20210522154537.pth"),
]

nets = [
    load_resnestnet(checkpoint_paths[0]),
    load_densenet(checkpoint_paths[1]),
    load_densenet(checkpoint_paths[2]),
    load_densenet(checkpoint_paths[3]),
    load_densenet(checkpoint_paths[4]),
    load_densenet(checkpoint_paths[5]),
    load_resnestnet(checkpoint_paths[6]),
    load_resnestnet(checkpoint_paths[7]),
    load_resnestnet(checkpoint_paths[8]),
    load_resnestnet(checkpoint_paths[9]),
    load_efficient_net(checkpoint_paths[10]),
]

pred_probas = predict(nets, test_data, names=False)
preds = [get_bird_names(get_thresh_preds(pred, thresh=THRESH)) for pred in pred_probas]
sub = preds_as_df(data, preds)
sub.to_csv("submission.csv", index=False)

In [ ]:
def get_metrics(s_true, s_pred):
    s_true = set(s_true.split())
    s_pred = set(s_pred.split())
    n, n_true, n_pred = len(s_true.intersection(s_pred)), len(s_true), len(s_pred)
    
    prec = n/n_pred
    rec = n/n_true
    f1 = 2*prec*rec/(prec + rec) if prec + rec else 0
    
    return {"f1": f1, "prec": prec, "rec": rec, "n_true": n_true, "n_pred": n_pred, "n": n}


if TARGET_PATH:
    sub_target = pd.read_csv(TARGET_PATH)
    sub_target = sub_target.merge(sub, how="left", on="row_id")
    assert sub_target["birds_x"].notnull().all()
    assert sub_target["birds_y"].notnull().all()
    
    df_metrics = pd.DataFrame([get_metrics(s_true, s_pred) for s_true, s_pred in zip(sub_target.birds_x, sub_target.birds_y)])

    print(df_metrics.mean())